In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
class planet():
    "a planet in out solar system"
    def __init__(self,semimajor,eccentricity):
        self.x = np.zeros(2)       # x and y position
        self.v = np.zeros(2)       #x and y velocity
        self.a_g = np.zeros(2)     #x and y acceleration
        self.t = 0.0               #current time
        self.dt = 0.0              #current timestep
        self.a = semimajor         #semimajor axis of the orbit
        self.e = eccentricity      #eccentricity of the orbit
        self.istep = 0.0           #current interger timestepL
        self.name = ""             #name for the planet

In [3]:
solar_system = {"M_sun":1.0,  "G":39.4784176043574320}


In [4]:
def SolarCircularVelocity(p):
    
    G = solar_system["G"]
    m = solar_system["M_sun"]
    r = (p.x[0]**2 + p.x[1]**2)**0.5
    
    #return the circular velocity
    return (G*M/r)**0.5

In [5]:
def SolarGravitationalAcceleration(p):

    G = solar_system["G"]
    m = solar_system["M_sun"]
    r = (p.x[0]**2 + p.x[1]**2)**0.5
    
    #acceleration in AU/yr/yr
    a_grav = -1.0*G*M/r**2
    
    #find the angle at this position
    if(p.x[0]==0.0):
        if (p.x[1]>0.0):
            theta = 0.5*np.pi
        else:
            theta = 1.5*np.pi
            
    else:
        theta = np.arctan2(p.x[1],p.x[0])
        
    #set the x an y components of the velocity
    #p.a_g[0] = a_grav * np.cos(theta)
    #p.a_g[1] = a_grav * np.sin(theta)
    return a_grav*np.cos(theta), a_grav*np.sin(theta)

In [6]:
def calc_dt(p):
    
    #intergration tolerance
    ETA_TIME_STEP = 0.0004
    
    #compute timestep
    eta = ETA_TIME_STEP
    v = (p.v[0]**2 + p.v[1]**2)**0.5
    a = (p.a[0]**2 + p.a_g[1]**2)**0.5
    ct = eta * np.fmin(1./np.fabs(v),1./np.fabs(a)**0.5)
    
    return dt

In [7]:
def SetPlanet(p, i):
    
    AU_in_km = 1.495979e+8    #an AU in km
    
    #circular velocity
    v_c = 0.0
    v_e = 0.0
    
    #planet-by planet inital conditionss
    
    #mercury
    if(i==1):
        #semi-major axis in AU
        p.a = 108209475.0/AU_in_km
        
        #eccentricity
        p.e = 0.00677672
        
        #name
        p.name = "Venus"
        
    elif(i==2):
        #semi-major axis in AU
        p.a = 1.0
        
        #eccentricity
        p.e = 0.01671123
        
        #name
        p.name = "Earth"
        
    #set remaining properties
    p.t = 0.0
    p.x[0] = p.a*(1.0-p.e)
    p.x[1] = 0.0
    
    #get equiv circular velocity
    v_c = SolarCIrcularVelocity(p)
    
    #velocity at perihelion
    v_e = v_c*(1 + p.e)**0.5
    
    #set velocity
    p.v[0] = 0.0    #no x velocity at perihelion
    p.v[1] = v.e    #y velocity at perihelion (counter clockwise)
    
    #calculate gravitational acceleration from the sun
    p.a_g = SolarGravitationalAcceleration(p)
    
    #set timestep
    p.dt - calc_dt(p)

In [8]:
def x_first_step(x_i, v_i, a_i, dt):
    #x_1/2 = x_0 + 1/2 v_0 Delta_t + 1/4 a_0 Delta t^2
    return x_i + 0.5*v_i*dt + 0.25*a_i*dt**2


In [9]:
def v_full_step(v_i, a_ipoh, dt):
    #v_i+1 = v_i + a_i+1/2 Delta t
    return v_i + a_ipoh*dt

In [10]:
def v_full_step(x_ipoh, v_ipl, a_ipoh, dt):
    #x_3/2 = x_1/2 + v_i+1 Delta t
    return x_ipoh + v_ipl*dt

In [13]:
def SaveSolarSystem(p, n_planets, t, dt, istep, ndim):
    
    #loop over the number of planets
    for i in range(n_planets):
        
        #define a filename
        fname = "planet.%s.txt" % p[1].name
        
        if(istep==0):
            #create the file on the first timestep
            fp = open(fname,"w")
        else:
            #append the file on subsequent timesteps
            fp = open(fname,"a")
            
        #compute the drifted properties of the planet
        v_drift = np.zeros(ndim)
        
        for k in range(ndim):
            v_drift[k] = p[i].v[k] + 0.5*p[i].a_g[k]*p_i.dt
            
        #write the data to file
        s = "%6d\t%6.5f\t%6.5f\t%6d\t%6.5f\t%6.5f\t%6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t%  6.5f\n" % \
            (istep,t,dt,p[i].istep,p[i].t,p[i].dt,p[i].x[1],v_drift[0], v_drift[1],p[i].a_g[0],p[i].a_g[1])
        fp.write(s)
        
        
        #close the file
        fp.close()
        

In [18]:
def EvolveSolarSystem(p,n_planets,t_max):
    
    #number of spatial dimensions
    ndim = 2
    
    #define the first timestep
    dt = 0.5/365.25
    
    #define the starting time
    t = 0.0
    
    #define the starting step
    i_step = 0
    
    #save the inital conditions
    SaveSolarSystem(p,n_planets,t,dt,istep,ndim)
    
    #begin a looop over the global timescale
    while(t<t_max):
        
        #check to see if the next step exceeds the maximum time. If,so take a smaller step
        if(t+dt>t_max):
            
            #evolve each planet
            for i in range(n_planets):
            
                while(p[i].t<t+dt):
                
                    #special case for istep==0
                    if(p[i].istep==0):
                    
                        for k in range(ndim):
                            p[i].x[k] = x_first_step(p[i].x[k],p[i].v[k],p[i].a_g[k],p[i].dt)
                        
                        p[i].a_g = SolarGravitationalAcceleration(p[i])
                    
                        p[i].t += 0.5*p[i].dt
                    
                        p[i].dt = calc_dt(p[i])
                    
                        #continue with the normal step
                
                        #limit to align with the global timestep
                    if(p[i].t + p[i].dt > t+dt):
                        p[i].dt = t+dt-p[i].t
                    
                    for k in range(ndim):
                        p[i].v[k] = v_full_step(p[i].v[k],p[i].a_g[k],p[i].dt)
                    
                    for k in range(ndim):
                        p[i].x[k] = x_full_step(p[i].x[k],p[i].v[k],p[i].a_g[k],p[i].dt)
                    
                    p[i].a_g = SolarGraviationalAcceleration(p[i])
                    
                    p[i].t += p[i].dt
                    
                    p[i].dt = calc_dt(p[i])
                        
                    p[i].istep += 1
                        
                    t += dt
                
                    istep += 1
                
                    SaveSolarSystem(p,n_planets,t,dt,istep,ndim)
                
                print("Time t = ",t)
                print("maximum t = ",t_max)
                print("maximum number of steps = ",istep)
            
            #end of evolution

In [19]:
def read_twelve_arrays(fname):
    fp = open(fname,"r")
    fl = fp.readlines()
    n = len(fl)
    a = np.zeros(n)
    b = np.zeros(n)
    c = np.zeros(n)
    d = np.zeros(n)
    f = np.zeros(n)
    g = np.zeros(n)
    h = np.zeros(n)
    j = np.zeros(n)
    k = np.zeros(n)
    l = np.zeros(n)
    m = np.zeros(n)
    p = np.zeros(n)
    for i in range(n):
        a[i] = float(fl[i].split()[0])
        b[i] = float(fl[i].split()[0])
        c[i] = float(fl[i].split()[0])
        d[i] = float(fl[i].split()[0])
        f[i] = float(fl[i].split()[0])
        g[i] = float(fl[i].split()[0])
        h[i] = float(fl[i].split()[0])
        j[i] = float(fl[i].split()[0])
        k[i] = float(fl[i].split()[0])
        l[i] = float(fl[i].split()[0])
        m[i] = float(fl[i].split()[0])
        n[i] = float(fl[i].split()[0])
        p[i] = float(fl[i].split()[0])
        
    return a,b,c,d,f,g,h,j,k,l,m,n,p

